<a href="https://colab.research.google.com/github/aksh98/Collaborative_Filtering/blob/master/CFAssignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Getting started with Colab**

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse


from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
# !mkdir -p drive
# !google-drive-ocamlfuse drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
%cd drive
%cd My Drive/Colab Notebooks 
%cd Assignments_sem7/


/content/drive
/content/drive/My Drive/Colab Notebooks
/content/drive/My Drive/Colab Notebooks/Assignments_sem7


In [0]:
!ls

Assignment1_CF	      Assignment3.ipynb     IP.ipynb
Assignment2_IA.ipynb  IA_Assignment1.ipynb  ml-100k


## **Recommendation System** 
I've implemented a recomendation system using KNN and Cosine similarity 

In [0]:
import pandas as pandas
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances
from numpy import newaxis
from sklearn.metrics import mean_absolute_error

# 100k ratings - 943 users - 1682 items
# each user rated atleast 20 movies
# user_id | item_id | rating | timestamp |



In [0]:
header = ['user','item','rating','time']


dataframe = pandas.read_csv('ml-100k/u.data',sep='\t',names=header)


Just some tables to show what the data looks like...


In [0]:
dataframe.shape

(100000, 4)

In [0]:
dataframe.head()

,user,item,rating,time
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [0]:
from sklearn.metrics import mean_absolute_error

dataframe = dataframe.drop(labels='time',axis=1)

In [0]:
dataframe.head()

,user,item,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [0]:
Num_users = 943
Num_mov = 1682

In [0]:

train,test = train_test_split(dataframe,test_size = 0.2,shuffle=False)
# algo = KNN()
train.shape,test.shape

((80000, 3), (20000, 3))

In [0]:
mat_train = np.zeros((Num_users,Num_mov))
mat_test = np.zeros((Num_users,Num_mov))
j = 0
for i in train.itertuples():
  mat_train[i[1]-1,i[2]-1] = i[3]
  if(j<5):
    print("train-> ",i[1]-1,i[2]-1,i[3])
  j+=1
print("---------")
j = 0
for i in test.itertuples():
  mat_test[i[1]-1,i[2]-1] = i[3]
  if(j<5):
    print("test -> ",i[1]-1,i[2]-1,i[3])
  j+=1
print(mat_train.shape)



train->  195 241 3
train->  185 301 3
train->  21 376 1
train->  243 50 2
train->  165 345 1
---------
test ->  862 1430 4
test ->  760 1286 1
test ->  862 321 1
test ->  827 693 2
test ->  888 522 4
(943, 1682)


Defining functions used for predictions and one is div0 which solves the problem of division with zero.

In [0]:
#increase dimension using newaxis
def div0( arr1,arr2 ):
  with np.errstate(divide='ignore', invalid='ignore'):
    final=np.true_divide(arr1,arr2)
    final[ ~ np.isfinite(final)] = 0  
  return final

def user_predict(cosine,rating):
  mean = rating.mean(axis=1)
  diff = (rating - mean[:,newaxis])
  #compute cosine distance
  a = np.array([np.sum(cosine,axis=1)]).T
  b = (cosine.dot(diff))
  cosine_sim = div0(b,a)
#   print(b.shape,a.shape,cosine_sim.shape,mean[:,newaxis].shape)
  predict = mean[:,newaxis] + cosine_sim
#   print("up = ",predict.shape) (943, 1682)
  return predict


def item_predict(cosine,rating):
  array = np.array([np.sum(cosine,axis=1)])
  a = rating.dot(cosine)
#   predict = np.nan_to_num(a / array)
  predict = div0(a,array)
#   predict[predict == np.inf] = 0
#   print("ip - ",a.shape,array.shape,predict.shape)
  return predict


Algorithm using Cosine distance to compute the similarity and predict the ratings.

The Mean Absolute Error:
- For items ->  0.296
- For Users ->  0.268

In [0]:
from sklearn.model_selection import KFold
# from pandas import dataframe as df
# import pandas.dataframe as df

def kfold_tt(train,test,mae_isum,mae_usum):
  mat_train = np.zeros((Num_users,Num_mov))
  mat_test = np.zeros((Num_users,Num_mov))
  j = 0
  for i in train.itertuples():
    mat_train[i[1]-1,i[2]-1] = i[3]
#     if(j<5):
#       print("train-> ",i[1]-1,i[2]-1,i[3])
#     j+=1
#   print("---------")
  j = 0
  for i in test.itertuples():
    mat_test[i[1]-1,i[2]-1] = i[3]
#     if(j<5):
#       print("test -> ",i[1]-1,i[2]-1,i[3])
    j+=1
#   print(mat_train.shape)
 
  user_matrix = cosine_similarity(mat_train)
  user_matrix.shape
  item_matrix = cosine_similarity(mat_train.T)
  item_matrix.shape  
  
  item_matrix = np.abs(item_matrix)
  user_matrix = abs(user_matrix)

  # print(np.isnan(item_matrix),np.isnan(mat_train))
  item_prediction = item_predict(item_matrix,mat_train)
  user_prediction = user_predict(user_matrix,mat_train)
  
  mae_u = mean_absolute_error(user_prediction,mat_test)
  mae_i = mean_absolute_error(item_prediction, mat_test)
#   print("User - ",mae_u)
#   print("item - ",mae_i)
  mae_isum += mae_i
  mae_usum += mae_u
  
  return mae_isum,mae_usum
  

kf = KFold(n_splits=5)
mae_isum,mae_usum = 0,0
for traini,testi in kf.split(dataframe):
  train = dataframe.iloc[traini]
  test = dataframe.iloc[testi]
  mae_isum,mae_usum = kfold_tt(train,test,mae_isum,mae_usum)
  
print("MAE_avg - ITEM -> ",mae_isum,"\nMAE_avg - USER -> ",mae_usum)




MAE_avg - ITEM ->  1.4796943065090207 
MAE_avg - USER ->  1.3422562317934714


Using KNN

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
# ========================================================

kf = KFold(n_splits=5)
mae_isum, mae_usum = 0,0

neighbour = KNeighborsClassifier(metric = 'cosine',n_neighbors=10)

for traini, testi in kf.split(dataframe):
  train = dataframe.iloc[traini]
  test = dataframe.iloc[testi]
  neighbour.fit(train.iloc[:,:1],train.iloc[:,2])
  pred = neighbour.predict(test.iloc[:,0:1])
  print(accuracy_score(test.iloc[:,2],pred))
  
#   neighbour.fit(data.iloc[:,0:4], data['Name'])
  
#   mae_isum,mae_usum = kfold_tt(train,test,mae_isum,mae_usum)
  
print("MAE_avg - ITEM -> ",mae_isum/5,"\nMAE_avg - USER -> ",mae_usum/5)


